In [ ]:
###瞳孔数据预处理第一步：删去置信度小于0.6的值
###/////////////////////////////////////////////////////////////////////////////////////////

import pandas as pd
import csv 

f=pd.read_csv(r'C:/Users/22064/Desktop/毕业设计/瞳孔数据/被试2瞳孔数据/2-2瞳孔数据.csv')
header = ('confidence','diameter')
with open('2-2_pupildata.csv', mode='a', newline='') as data_file:
    data_writer = csv.writer(data_file)
    data_writer.writerow(header)
    for i in range (f.shape[0]): 
        if f.iloc[i,0]>=0.6:
            data_writer.writerow([f.iloc[i,0],f.iloc[i,1]])

In [ ]:
###瞳孔数据预处理第二步：移动平均滤波
###/////////////////////////////////////////////////////////////////////////////////////////

import pandas as pd
import numpy as np
import csv
def moving_average_filter(data, window_size):
    """定义移动平均滤波器函数，输入数据和窗口大小"""
    window = np.ones(int(window_size))/float(window_size)
    """创建窗口，窗口大小为window_size，元素值为1/window_size"""
    return np.convolve(data, window, 'valid')
    """使用np.convolve函数进行卷积操作，返回输出信号"""
f=pd.read_csv(r'C:/Users/22064/Desktop/毕业设计/瞳孔数据/被试2瞳孔数据1.0/2-2_pupildata.csv')
signal=f.iloc[:,1]
# 应用移动平均滤波器
filtered_signal = moving_average_filter(signal, 1000)

filtered_signal = np.reshape(filtered_signal, (-1, 1))
header=('diameter','')
with open('2-2_pupil.csv', mode='a', newline='') as data_file:
    data_writer = csv.writer(data_file)
    data_writer.writerow(header)
    data_writer.writerows(filtered_signal)

In [ ]:
###基于多分类支持向量机识别认知模式
###//////////////////////////////////////////////////////////////////////////////////////////
# 导入所需库
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
train_data=pd.read_csv(r'C:/Users/22064/Desktop/毕业设计/生成的数据.csv')
X=train_data.iloc[:,0:4]
y=train_data.iloc[:,4]
# 划分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
svm=SVC(kernel='linear',C=100,gamma=0.01) #构建svm分类器
svm.fit(X_train,y_train) #模型训练
print("训练集上准确度为：{:.2f}".format(svm.score(X_train,y_train)))
print("测试集上准确度为：{:.2f}".format(svm.score(X_test,y_test)))

#混淆矩阵

from sklearn.metrics import classification_report
y_pred = svm.predict(X_test)
#print(classification_report(y_test,y_pred))

with open('classification_report.txt', 'w') as f:
    f.write(classification_report(y_test,y_pred))

import seaborn as sns; sns.set()
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test,y_pred)
#labels= train_data['mode'].unique()#获取类别
labels =['认知难度1','认知难度2','认知难度3','认知难度4','认知难度5','认知难度6','认知难度7','认知难度8',
                '认知难度9','认知难度10','认知难度11','认知难度12','认知难度13','认知难度14','认知难度15',
                '认知难度16','认知难度17','认知难度18','认知难度19','认知难度20']
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.figure(figsize=(9, 9))
sns.heatmap(mat.T, annot = True, cmap='Blues',fmt='d', cbar=True, xticklabels = labels ,yticklabels = labels)
plt.xlabel('true mode')
plt.ylabel('predicted mode')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.svg', format='svg')
plt.show()

In [ ]:
# 网格搜索交叉验证，返回最佳模型及其参数
from sklearn.model_selection import GridSearchCV
svm=SVC()
param_grid= {'kernel':['linear', 'rbf'], 'C':[0.1,1,10,100], 'gamma':[0.01,0.1,1]}
grid = GridSearchCV(svm,param_grid,cv=3)
grid.fit(X_train,y_train)
print(grid.best_params_) # 输出最佳参数组合
svm = grid.best_estimator_
print("训练集上准确度为：{:.2f}".format(svm.score(X_train,y_train)))
print("测试集上得分：{:.2f}".format(svm.score(X_test,y_test)))
# 在测试集上进行预测
y_pred = svm.predict(X_test)
# 计算分类准确度
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)